# Tabnet 1 - Modelo antes de entrar a la universidad

In [54]:
!pip install pytorch-tabnet
!pip install tabnet
!pip install wget

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [55]:
dfl=pd.read_csv("C:/Users/Profesor/data_imputation_paramodel1_train.txt",header=0, sep=",",na_values='\t', encoding='latin-1') #train
dfl_valid=pd.read_csv("C:/Users/Profesor/data_imputation_paramodel1_valid.txt",header=0, sep=",",na_values='\t', encoding='latin-1')
dfl_test=pd.read_csv("C:/Users/Profesor/data_imputation_paramodel1_test.txt",header=0, sep=",",na_values='\t', encoding='latin-1')


In [56]:
#Agregar una nueva col con el set al que pertenecen 
dfl["Set"]="train"
dfl_valid["Set"]="valid"
dfl_test["Set"]="test"

In [57]:
#Concatenando
dfl=dfl.append(dfl_valid)
dfl=dfl.append(dfl_test)

In [58]:
target = 'GRAD'
#if "Set" not in df.columns:
#    df["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df.shape[0],))

train_indices = dfl[dfl.Set=="train"].index
valid_indices = dfl[dfl.Set=="valid"].index
test_indices = dfl[dfl.Set=="test"].index

In [59]:
dfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23285 entries, 0 to 1897
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GRAD              23285 non-null  object 
 1   Year              23285 non-null  int64  
 2   Faculty           23285 non-null  object 
 3   Major             23285 non-null  object 
 4   Apt_Verbal        23285 non-null  int64  
 5   Aprov_Matem       23285 non-null  int64  
 6   Apt_Matem         23285 non-null  int64  
 7   Aprov_Espanol     23285 non-null  int64  
 8   Aprov_Ingles      23285 non-null  int64  
 9   INGRESO_FAMILIAR  23285 non-null  object 
 10  EDUC_PADRE        23285 non-null  object 
 11  EDUC_MADRE        23285 non-null  object 
 12  Gender            23285 non-null  object 
 13  School_Type       23285 non-null  object 
 14  Rel_Stud_GPA      23285 non-null  float64
 15  Set               23285 non-null  object 
dtypes: float64(1), int64(6), object(9)
memory

In [60]:
nunique = dfl.nunique()
types = dfl.dtypes

categorical_columns = []
categorical_dims =  {}
for col in dfl.columns:
    if types[col] == 'object' or nunique[col] < 200:  #The nunique() function is used to count distinct observations over requested axis.
        print(col, dfl[col].nunique())
        l_enc = LabelEncoder()
        dfl[col] = dfl[col].fillna("VV_likely") 
        dfl[col] = l_enc.fit_transform(dfl[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        dfl.fillna(dfl.loc[train_indices, col].mean(), inplace=True)

GRAD 2
Year 12
Faculty 5
Major 54
INGRESO_FAMILIAR 5
EDUC_PADRE 6
EDUC_MADRE 6
Gender 2
School_Type 3
Set 3


In [61]:
dfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23285 entries, 0 to 1897
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GRAD              23285 non-null  int32  
 1   Year              23285 non-null  int64  
 2   Faculty           23285 non-null  int32  
 3   Major             23285 non-null  int32  
 4   Apt_Verbal        23285 non-null  int64  
 5   Aprov_Matem       23285 non-null  int64  
 6   Apt_Matem         23285 non-null  int64  
 7   Aprov_Espanol     23285 non-null  int64  
 8   Aprov_Ingles      23285 non-null  int64  
 9   INGRESO_FAMILIAR  23285 non-null  int32  
 10  EDUC_PADRE        23285 non-null  int32  
 11  EDUC_MADRE        23285 non-null  int32  
 12  Gender            23285 non-null  int32  
 13  School_Type       23285 non-null  int32  
 14  Rel_Stud_GPA      23285 non-null  float64
 15  Set               23285 non-null  int32  
dtypes: float64(1), int32(9), int64(6)
memory 

In [62]:
unused_feat = ['Set']

features = [ col for col in dfl.columns if col not in unused_feat+[target]] #Select features without target and Set.

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns] #Select features without target and Set in "features" is categorical.

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns] #Select features without target and Set in "features" is categorical with count distinct observations.

In [63]:
dfl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23285 entries, 0 to 1897
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GRAD              23285 non-null  int32  
 1   Year              23285 non-null  int64  
 2   Faculty           23285 non-null  int32  
 3   Major             23285 non-null  int32  
 4   Apt_Verbal        23285 non-null  int64  
 5   Aprov_Matem       23285 non-null  int64  
 6   Apt_Matem         23285 non-null  int64  
 7   Aprov_Espanol     23285 non-null  int64  
 8   Aprov_Ingles      23285 non-null  int64  
 9   INGRESO_FAMILIAR  23285 non-null  int32  
 10  EDUC_PADRE        23285 non-null  int32  
 11  EDUC_MADRE        23285 non-null  int32  
 12  Gender            23285 non-null  int32  
 13  School_Type       23285 non-null  int32  
 14  Rel_Stud_GPA      23285 non-null  float64
 15  Set               23285 non-null  int32  
dtypes: float64(1), int32(9), int64(6)
memory 

In [64]:
dfl[features] = dfl[features].apply(lambda x: 1/(1+ np.exp(-x)), axis=0)  #Softmax Normalization

X_train = dfl[features].values[train_indices]
y_train = dfl[target].values[train_indices]

X_valid = dfl[features].values[valid_indices]
y_valid = dfl[target].values[valid_indices]

X_test = dfl[features].values[test_indices]
y_test = dfl[target].values[test_indices]

# Tuning

In [65]:
#hyperparameters

#optimizer_params_list=dict(lr=[2e-2,1])

max_epochs_list = [20,21,22,23,24,25,26,27,28,29,30]
batch_size_list = [1024,1025,1026,1027,1028,1029,1030]
virtual_batch_size_list = [128, 129, 130,131, 132,133, 134]
patience_list = [20,21,22,23,24,25,26]

In [66]:
def tabnet_grid_search(max_epochs,batch_size,virtual_batch_size,patience):
  model = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1
                      )
  model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=max_epochs , patience=patience,
    batch_size=batch_size, virtual_batch_size=virtual_batch_size,
    num_workers=0,
    weights=1,
    drop_last=False
    )
  Report = classification_report(y_test,model.predict(X_test),labels=[0,1], output_dict=True)
  return ([max_epochs,batch_size,virtual_batch_size,patience,Report['0']['recall']])

In [67]:
#Loop
result_list = []

for max_epochs in max_epochs_list:
  for batch_size in batch_size_list:
    for virtual_batch_size in virtual_batch_size_list:
      for patience in patience_list:
        result_list.append(tabnet_grid_search(max_epochs,batch_size,virtual_batch_size,patience))

Result_tabnet = pd.DataFrame(result_list, columns=['max_epochs','batch_size','virtual_batch_size','patience', 'Recall'])

Device used : cpu
epoch 0  | loss: 0.72718 | train_auc: 0.47715 | valid_auc: 0.48549 |  0:00:01s
epoch 1  | loss: 0.69584 | train_auc: 0.47682 | valid_auc: 0.48607 |  0:00:03s
epoch 2  | loss: 0.69442 | train_auc: 0.50351 | valid_auc: 0.51548 |  0:00:05s
epoch 3  | loss: 0.69379 | train_auc: 0.52779 | valid_auc: 0.533   |  0:00:06s
epoch 4  | loss: 0.6948  | train_auc: 0.47662 | valid_auc: 0.48169 |  0:00:08s
epoch 5  | loss: 0.69292 | train_auc: 0.47406 | valid_auc: 0.47291 |  0:00:09s
epoch 6  | loss: 0.69253 | train_auc: 0.47622 | valid_auc: 0.48651 |  0:00:11s
epoch 7  | loss: 0.69356 | train_auc: 0.50145 | valid_auc: 0.5012  |  0:00:13s
epoch 8  | loss: 0.69287 | train_auc: 0.47793 | valid_auc: 0.48807 |  0:00:14s
epoch 9  | loss: 0.69233 | train_auc: 0.51657 | valid_auc: 0.51097 |  0:00:16s
epoch 10 | loss: 0.69338 | train_auc: 0.51793 | valid_auc: 0.51498 |  0:00:18s
epoch 11 | loss: 0.69236 | train_auc: 0.47418 | valid_auc: 0.47602 |  0:00:19s
epoch 12 | loss: 0.69364 | train_a

C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.72718 | train_auc: 0.47715 | valid_auc: 0.48549 |  0:00:01s
epoch 1  | loss: 0.69584 | train_auc: 0.47682 | valid_auc: 0.48607 |  0:00:03s
epoch 2  | loss: 0.69442 | train_auc: 0.50351 | valid_auc: 0.51548 |  0:00:05s
epoch 3  | loss: 0.69379 | train_auc: 0.52779 | valid_auc: 0.533   |  0:00:07s
epoch 4  | loss: 0.6948  | train_auc: 0.47662 | valid_auc: 0.48169 |  0:00:09s
epoch 5  | loss: 0.69292 | train_auc: 0.47406 | valid_auc: 0.47291 |  0:00:10s
epoch 6  | loss: 0.69253 | train_auc: 0.47622 | valid_auc: 0.48651 |  0:00:12s
epoch 7  | loss: 0.69356 | train_auc: 0.50145 | valid_auc: 0.5012  |  0:00:14s
epoch 8  | loss: 0.69287 | train_auc: 0.47793 | valid_auc: 0.48807 |  0:00:16s
epoch 9  | loss: 0.69233 | train_auc: 0.51657 | valid_auc: 0.51097 |  0:00:17s
epoch 10 | loss: 0.69338 | train_auc: 0.51793 | valid_auc: 0.51498 |  0:00:19s
epoch 11 | loss: 0.69236 | train_auc: 0.47418 | valid_auc: 0.47602 |  0:00:21s
epoch 12 | loss: 0.69364 | train_a

C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.72718 | train_auc: 0.47715 | valid_auc: 0.48549 |  0:00:01s
epoch 1  | loss: 0.69584 | train_auc: 0.47682 | valid_auc: 0.48607 |  0:00:03s
epoch 2  | loss: 0.69442 | train_auc: 0.50351 | valid_auc: 0.51548 |  0:00:05s
epoch 3  | loss: 0.69379 | train_auc: 0.52779 | valid_auc: 0.533   |  0:00:07s
epoch 4  | loss: 0.6948  | train_auc: 0.47662 | valid_auc: 0.48169 |  0:00:09s
epoch 5  | loss: 0.69292 | train_auc: 0.47406 | valid_auc: 0.47291 |  0:00:10s
epoch 6  | loss: 0.69253 | train_auc: 0.47622 | valid_auc: 0.48651 |  0:00:12s
epoch 7  | loss: 0.69356 | train_auc: 0.50145 | valid_auc: 0.5012  |  0:00:14s
epoch 8  | loss: 0.69287 | train_auc: 0.47793 | valid_auc: 0.48807 |  0:00:16s
epoch 9  | loss: 0.69233 | train_auc: 0.51657 | valid_auc: 0.51097 |  0:00:18s
epoch 10 | loss: 0.69338 | train_auc: 0.51793 | valid_auc: 0.51498 |  0:00:20s
epoch 11 | loss: 0.69236 | train_auc: 0.47418 | valid_auc: 0.47602 |  0:00:21s
epoch 12 | loss: 0.69364 | train_a

...

C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.72718 | train_auc: 0.47715 | valid_auc: 0.48549 |  0:00:01s
epoch 1  | loss: 0.69584 | train_auc: 0.47682 | valid_auc: 0.48607 |  0:00:03s
epoch 2  | loss: 0.69442 | train_auc: 0.50351 | valid_auc: 0.51548 |  0:00:05s
epoch 3  | loss: 0.69379 | train_auc: 0.52779 | valid_auc: 0.533   |  0:00:07s
epoch 4  | loss: 0.6948  | train_auc: 0.47662 | valid_auc: 0.48169 |  0:00:09s
epoch 5  | loss: 0.69292 | train_auc: 0.47406 | valid_auc: 0.47291 |  0:00:10s
epoch 6  | loss: 0.69253 | train_auc: 0.47622 | valid_auc: 0.48651 |  0:00:12s
epoch 7  | loss: 0.69356 | train_auc: 0.50145 | valid_auc: 0.5012  |  0:00:14s
epoch 8  | loss: 0.69287 | train_auc: 0.47793 | valid_auc: 0.48807 |  0:00:16s
epoch 9  | loss: 0.69233 | train_auc: 0.51657 | valid_auc: 0.51097 |  0:00:18s
epoch 10 | loss: 0.69338 | train_auc: 0.51793 | valid_auc: 0.51498 |  0:00:20s
epoch 11 | loss: 0.69236 | train_auc: 0.47418 | valid_auc: 0.47602 |  0:00:22s
epoch 12 | loss: 0.69364 | train_a

C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Profesor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Device used : cpu
epoch 0  | loss: 0.72718 | train_auc: 0.47715 | valid_auc: 0.48549 |  0:00:01s
epoch 1  | loss: 0.69584 | train_auc: 0.47682 | valid_auc: 0.48607 |  0:00:03s
epoch 2  | loss: 0.69442 | train_auc: 0.50351 | valid_auc: 0.51548 |  0:00:05s
epoch 3  | loss: 0.69379 | train_auc: 0.52779 | valid_auc: 0.533   |  0:00:07s
epoch 4  | loss: 0.6948  | train_auc: 0.47662 | valid_auc: 0.48169 |  0:00:09s
epoch 5  | loss: 0.69292 | train_auc: 0.47406 | valid_auc: 0.47291 |  0:00:12s
epoch 6  | loss: 0.69253 | train_auc: 0.47622 | valid_auc: 0.48651 |  0:00:14s
epoch 7  | loss: 0.69356 | train_auc: 0.50145 | valid_auc: 0.5012  |  0:00:16s
epoch 8  | loss: 0.69287 | train_auc: 0.47793 | valid_auc: 0.48807 |  0:00:18s
epoch 9  | loss: 0.69233 | train_auc: 0.51657 | valid_auc: 0.51097 |  0:00:20s
epoch 10 | loss: 0.69338 | train_auc: 0.51793 | valid_auc: 0.51498 |  0:00:22s
epoch 11 | loss: 0.69236 | train_auc: 0.47418 | valid_auc: 0.47602 |  0:00:23s


KeyboardInterrupt: 

In [68]:
Result_tabnet = pd.DataFrame(result_list, columns=['max_epochs','batch_size','virtual_batch_size','patience', 'Recall'])
Result_tabnet.sort_values(by=['Recall'], inplace=True, ascending=False)
print(Result_tabnet)

     max_epochs  batch_size  virtual_batch_size  patience  Recall
0            20        1024                 128        20     1.0
325          20        1030                 132        23     1.0
319          20        1030                 131        24     1.0
320          20        1030                 131        25     1.0
321          20        1030                 131        26     1.0
..          ...         ...                 ...       ...     ...
270          20        1029                 131        24     0.0
269          20        1029                 131        23     0.0
268          20        1029                 131        22     0.0
267          20        1029                 131        21     0.0
219          20        1028                 131        22     0.0

[369 rows x 5 columns]


In [69]:
Result_tabnet[Result_tabnet['Recall']<0.94]

,max_epochs,batch_size,virtual_batch_size,patience,Recall
104,20,1026,128,26,0.785194
103,20,1026,128,25,0.785194
101,20,1026,128,23,0.785194
100,20,1026,128,22,0.785194
99,20,1026,128,21,0.785194
...,...,...,...,...,...
270,20,1029,131,24,0.000000
269,20,1029,131,23,0.000000
268,20,1029,131,22,0.000000
267,20,1029,131,21,0.000000


In [82]:
M=Result_tabnet[Result_tabnet['Recall']<0.80]

In [83]:
M[0:50]

,max_epochs,batch_size,virtual_batch_size,patience,Recall
104,20,1026,128,26,0.785194
103,20,1026,128,25,0.785194
101,20,1026,128,23,0.785194
100,20,1026,128,22,0.785194
99,20,1026,128,21,0.785194
98,20,1026,128,20,0.785194
102,20,1026,128,24,0.785194
300,20,1030,128,26,0.658981
299,20,1030,128,25,0.658981
298,20,1030,128,24,0.658981


# Modelo luego del primer año

In [89]:
clf_1= TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=1,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.95},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

clf_1.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['auc'],
    max_epochs=20 , patience=25,
    batch_size=1030, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
) 

Device used : cpu
epoch 0  | loss: 0.71434 | train_auc: 0.47285 | valid_auc: 0.46765 |  0:00:01s
epoch 1  | loss: 0.69872 | train_auc: 0.52226 | valid_auc: 0.51371 |  0:00:03s
epoch 2  | loss: 0.69465 | train_auc: 0.47401 | valid_auc: 0.47404 |  0:00:04s
epoch 3  | loss: 0.69432 | train_auc: 0.52389 | valid_auc: 0.51477 |  0:00:06s
epoch 4  | loss: 0.6957  | train_auc: 0.52164 | valid_auc: 0.51533 |  0:00:08s
epoch 5  | loss: 0.69408 | train_auc: 0.52272 | valid_auc: 0.51813 |  0:00:09s
epoch 6  | loss: 0.69408 | train_auc: 0.52691 | valid_auc: 0.52773 |  0:00:11s
epoch 7  | loss: 0.69437 | train_auc: 0.47555 | valid_auc: 0.46984 |  0:00:13s
epoch 8  | loss: 0.69366 | train_auc: 0.47515 | valid_auc: 0.47251 |  0:00:14s
epoch 9  | loss: 0.6937  | train_auc: 0.5245  | valid_auc: 0.52663 |  0:00:16s
epoch 10 | loss: 0.69444 | train_auc: 0.52768 | valid_auc: 0.52437 |  0:00:18s
epoch 11 | loss: 0.69287 | train_auc: 0.52344 | valid_auc: 0.51199 |  0:00:19s
epoch 12 | loss: 0.69433 | train_a

In [90]:
Report = classification_report(y_test,clf_1.predict(X_test),labels=[0,1])
print('Classification report : \n',Report)

Classification report : 
               precision    recall  f1-score   support

           0       0.47      0.34      0.39       824
           1       0.58      0.70      0.63      1074

    accuracy                           0.54      1898
   macro avg       0.52      0.52      0.51      1898
weighted avg       0.53      0.54      0.53      1898



In [91]:
dataset_name = 'data_pre'
preds = clf_1.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)
print(f"AUC : {test_auc}")

AUC : 0.5304262488474263


In [92]:
pd.crosstab(clf_1.predict(X_test), y_test, 
            rownames=['pred'], 
            colnames=['Actual'], margins=False, margins_name="Total")

Actual,0,1
pred,,
0,281,322
1,543,752


In [93]:
classification_report(y_test,clf_1.predict(X_test),labels=[0,1], output_dict=True)['0']['recall']

0.34101941747572817

In [94]:
classification_report(y_test,clf_1.predict(X_test),labels=[0,1], output_dict=True)['0']['f1-score']

0.39383321653819203

In [95]:
preds = clf_1.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)
print(f"AUC : {test_auc}")

AUC : 0.5304262488474263


In [96]:
1-classification_report(y_test,clf_1.predict(X_test),labels=[0,1], output_dict=True)['0']['recall']

0.6589805825242718

In [97]:
543/(543+752+322+281)

0.28609062170706007

In [ ]:
0.6513157894736842-1
